In [1]:
import bz2
import nltk
import numpy as np
import json
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
train_file = bz2.BZ2File('../input_data/train.json.bz2')
test_file = bz2.BZ2File('../input_data/test.json.bz2')
train_file = json.load(train_file)
test_file = json.load(test_file)
print(len(train_file))
print(len(test_file))

618911
125712


In [12]:
#def process_input_file(input_file, valid_links, link_delim):
#    result_file = []
#    links = []
#    for i, line in enumerate(input_file):
#        sentence, link = next(iter(line.items()))
#        sentence = sentence.lower().strip()
#        if link is None or link not in valid_links:
#            result_file.append(sentence)
#        else:
#            for sub_link in sentence.split(' '):
#                result_file.append(sub_link)
#                result_file[-1] = link_delim + result_file[-1] + link_delim
#                links.append(link)
#    return result_file, links


def process_input_file(input_file, valid_links, link_delim):
    links = []
    for i, line in enumerate(input_file):
        sentence, link = next(iter(line.items()))
        sentence = sentence.lower().strip()
        if link not in valid_links:
            input_file[i] = sentence
        else:
            sub_links = sentence.split()
            input_file[i] = ""
            for sub_link in sub_links:
                input_file[i] += link_delim + sub_link + link_delim + " "
                links.append(link)
            input_file[i] = input_file[i][:-1]
    return input_file, links

def encode_labels(sentences_with_link_markup, links, link2idx, link_delim):
    encoded_labels = []
    link_index = 0;
    for sentence in sentences_with_link_markup:
        encoded_label = []
        for word in sentence:
            if (word[0] == link_delim and word[-1] == link_delim):
                encoded_label.append(link2idx[links[link_index]])
                link_index += 1
            else:
                encoded_label.append(0)
        encoded_labels.append(encoded_label)
    return encoded_labels

def extend_sublink_mappings(link2idx):
    for link,index in list(link2idx.items()):
        sub_links = link.split(" ")
        for sub_link in sub_links:
            link2idx[sub_link] = index
    return link2idx

In [13]:
# Preprocess data ...

LINK_DELIM = '_'
link_treshold = 1
valid_links = Counter([next(iter(sentence.values())) for sentence in train_file if next(iter(sentence.values())) is not None])
valid_links = set([link for link,frequence in valid_links.items() if frequence >= link_treshold])

train_file, train_links = process_input_file(train_file, valid_links, LINK_DELIM)
test_file, test_links = process_input_file(test_file, valid_links, LINK_DELIM)

train_file = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(" ".join(train_file))]
test_file = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(" ".join(test_file))]
train_sentences = [[word.replace(LINK_DELIM,'') for word in sentence] for sentence in train_file]
test_sentences = [[word.replace(LINK_DELIM,'') for word in sentence] for sentence in test_file]

vocabulary = Counter([word for sentence in train_sentences for word in sentence for char in word])
vocabulary = ['_PAD','_UNK'] + sorted(vocabulary, key=vocabulary.get, reverse=True)
output = ["_TEXT"] + train_links

word2idx = {w:i for i,w in enumerate(vocabulary)}
idx2word = {i:w for i,w in enumerate(vocabulary)} # probably vocabulary array is enough
link2idx = {l:i for i,l in enumerate(output)}
link2idx = extend_sublink_mappings(link2idx)

train_encoded_labels = encode_labels(train_file, train_links, link2idx, LINK_DELIM)
test_encoded_labels = encode_labels(test_file, test_links, link2idx, LINK_DELIM)
print(len(train_links))


IndexError: list index out of range

In [10]:
#TODO: check double dominoes in
# print(train_file[y])
# and the reason why process_input_file crashed when returning a larger file

#train_encoded_labels = encode_labels(train_file, train_links, link2idx, LINK_DELIM)

#print([word for sentence in train_file for word in sentence if word[0] == '_' == word[-1]][:50])
#y=5
print(len([word for sentence in train_file for word in sentence if word[0]==word[-1]=='_']))
print(len(train_links))
print("---")
#print([x for x in train_links if "sermon" in x.lower()])
#print("sermon" in map(str.lower, list(link2idx.keys())))
y=2
print(train_encoded_labels[y])
print(len(train_encoded_labels[y]))
print(train_file[y])
print(train_sentences[y])
print(len(train_sentences[y]))
print(output[157700])
#print(output[202145])
#print(link2idx["Polytope"])

        
train_sentences_len = [len(sentence) for sentence in train_sentences]
max_train_sentences_len = max(train_sentences_len)

test_sentences_len = [len(sentence) for sentence in test_sentences]
max_test_sentences_len = max(test_sentences_len)


655011
654802
---


NameError: name 'train_encoded_labels' is not defined

In [ ]:
# Encode words as integers ...

for i, sentence in enumerate(train_sentences):
    train_sentences[i] = [word2idx[word] if word in word2idx else 1 for word in sentence]
    
for i, sentence in enumerate(test_sentences):
    test_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

In [ ]:
print([idx2word[x] for x in train_sentences[0]])
print([x for x in train_file[0]])